In [1]:
import pandas as pd

In [2]:
datasets = [ 'kdd99', 'kdd_dropped', 'creditcardfraud', 'ecoli', 'optical_digits', 'satimage', 'pen_digits', 'abalone', 'sick_euthyroid',
              'spectrometer', 'car_eval_34', 'isolet', 'us_crime', 'yeast_ml8', 'scene', 'libras_move', 'thyroid_sick', 'coil_2000',
              'arrhythmia', 'solar_flare_m0', 'oil', 'car_eval_4', 'wine_quality', 'letter_img', 'yeast_me2', 'webpage',
              'ozone_level', 'mammography', 'protein_homo', 'abalone_19' ]
len(datasets)

30

In [3]:
# latex用のため，`_`を`\_`に置き換え
datasets_replaced_ = [c.replace('_', r'\_') for c in datasets]

In [7]:
from visualization.latex_table import LatexTable, MultiColumn

In [21]:
_optuna = False
_preprocessing = 'none'  # none, ss, nl
_ae_preprocessing = 'none'
layers = [
    [0],
    [20, 10, 5],
    [20, 15, 10],
    [20, 15, 10, 5],
]
models = [
    'Logistic Regression',
    'SVM',
    'Random Forest',
    'LightGBM',
    'Multi Perceptron'
]
mdls = ['lr', 'svm', 'rf', 'lgb', 'mp']
used_classes = ['all', 'minority', 'majority']

use_model = models[0]
use_mdl = mdls[0]
use_class = used_classes[0]
standardize = False
normalize = False
ae_standardize = False
ae_normalize = False

table = LatexTable(
    f"{use_model},Optuna:{_optuna},前処理:{_preprocessing},AE前処理:{_ae_preprocessing}", 
    f"{use_mdl}&optuna-{_optuna}&preprocessing-{_preprocessing}&ae-preprocessing-{_ae_preprocessing}",
    r"l*{8}{c}",
    column_num=9
    )


In [22]:
from db_query import fetch_latest_record


table.add_columns(["optuna", MultiColumn(_optuna, 8)])
table.add_columns(["preprocessing", MultiColumn(_preprocessing, 8)])
table.add_columns(["ae-preprocessing", MultiColumn(_ae_preprocessing, 8)])
table.add_columns(["layers", 'none', *layers[1:4], 'none', *layers[1:4]])
table.add_hline()
table.add_columns(
    ['Dataset', MultiColumn('minority F-accuracy', 4),
                MultiColumn('macro F-accuracy', 4),])
table.add_hline()
for i in range(len(datasets)):
    dataset = datasets[i]
    cols = [datasets_replaced_[i]]
    res_minority = []
    res_macro = []
    for layer in layers:
        record = fetch_latest_record({
            'dataset.name': dataset,
            'dataset.standardization': standardize,
            'dataset.normalization': normalize,
            'model.name': use_mdl,
            'model.optuna': _optuna,
            'ae.layers': layer,
            'ae.used_class': use_class,
            'ae.standardization': ae_standardize,
            'ae.normalization': ae_normalize,
            },
        )
        if record is None:
            res_minority.append('-')
            res_macro.append('-')
        else:
            res_minority.append(f"{record['result']['minority']['f1']:.3f}")
            res_macro.append(f"{record['result']['macro']['f1']:.3f}")
    cols.extend(res_minority)
    cols.extend(res_macro)


    table.add_columns(cols)
cnt = 0
with open(f'thesis/tables/table{cnt}.tex', 'w') as f:
    cnt += 1
    f.write(table.compile())
